<a href="https://colab.research.google.com/github/DastanIqbal/LearnOpenCV/blob/main/src/OpenCV/dlap/week6/ApplicationApperialMiniDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download the dataset
!wget -O mini-dataset.zip "https://www.dropbox.com/sh/dmjj4tfz3qk4zpu/AACy-H1cHYsEpkP94p9W6hvQa?dl=1"

# Extract the dataset
!unzip -q -u mini-dataset.zip

In [ ]:
# Importing Libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Softmax, Dense,Dropout,BatchNormalization,Flatten
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

**Loading Data**

In [ ]:
# Defining Data Generator
gen = ImageDataGenerator()
# Defining  Batch Size and seed
BATCH_SIZE = 8
SEED = 21

In [ ]:
# Acquring Training images from the directory using flow_from_directory() method.
train_datagen = gen.flow_from_directory(directory = '/content/train',
                                        target_size = (224, 224),
                                        color_mode = "rgb",
                                        classes = None,
                                        class_mode = "categorical",
                                        batch_size = BATCH_SIZE,
                                        shuffle = True,
                                        seed = SEED,
                                        interpolation = "nearest")

In [ ]:
# Acquring Validation images from the directory using flow_from_directory() method.
val_datagen = gen.flow_from_directory(directory = '/content/val',
                                      target_size = (224, 224),
                                      color_mode = "rgb",
                                      classes = None,
                                      class_mode = "categorical",
                                      batch_size = BATCH_SIZE,
                                      shuffle = True,
                                      seed = SEED,
                                      interpolation = "nearest")

In [ ]:
# Acquring Testing images from the directory using flow_from_directory() method.
test_datagen = gen.flow_from_directory(directory = '/content/test',
                                       target_size = (224, 224),
                                       color_mode = "rgb",
                                       classes = None,
                                       class_mode = "categorical",
                                       batch_size = 1,
                                       shuffle = False, # Here Shuffle is set to False, so that we can test our model.
                                       seed = SEED,
                                       interpolation = "nearest

In [ ]:
# Classes of images in test dataset
classes = ['Dress', 'Pants', 'Shirt', 'Shoes', 'Shorts']

In [ ]:
# Preparing the Samples and Plot for displaying output

# Create figure
fig = plt.figure(figsize = (12, 12))
for i in range(9):
  plt.subplot(330 + 1 + i)
  img, label = next(train_datagen)

  label = label[0].astype('uint8')
  label = np.squeeze(label)
  label = np.argmax(label, axis = 0)

  plt.axis('off')
  plt.imshow(img[0].astype(np.uint8))
  plt.title(classes[label])

# Displaying the figure
plt.show()

**Initialize Pre-trained Model**

In [ ]:
# Importing EfficientNet B7 Model
image_model = tf.keras.applications.EfficientNetB7(include_top = False,
                                                   weights ='imagenet',
                                                   input_shape = (224, 224, 3),
                                                   pooling = 'max')


In [ ]:
# Printing Model Summary
print(image_model.summary())

In [ ]:
# Adding Dense, BatchNorm and Droupout layers to base model
# to have output for 5 Class Classification
x = Dense(1024, activation = 'relu')((image_model.output))
x = Dropout(0.5)(x)
x = Dense(512, activation = 'relu')(x)
x = Dense(64, activation = 'relu')(x)
x = BatchNormalization()(x)
x = Dense(16, activation = 'relu')(x)
predictions = Dense(5, activation = 'softmax')(x)

# Define the input and output layers of the model
model = Model(inputs = image_model.input, outputs = predictions)

# Compile model and define Optimizer
model.compile(optimizer = tf.keras.optimizers.Adam(0.0001),
              loss = "categorical_crossentropy",
              metrics = ["accuracy"])

In [ ]:
# Printing Final Model Summary
model.summary()

**Callbacks**

In [ ]:
# Assigning Checkpoint Path for Saved Model
filepath = '/content/Models/apparel-classification-mini-dataset-EfNetB7_15Epochs.h5'

In [ ]:
# Defining ModelCheckpoint Callback
model_save = tf.keras.callbacks.ModelCheckpoint(filepath,
                                                monitor = "val_accuracy",
                                                verbose = 0,
                                                save_best_only = True,
                                                save_weights_only = False,
                                                mode = "max",
                                                save_freq = "epoch")

In [ ]:
# Defining Reduce lr callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss',
                                                 factor = 0.1,
                                                 patience = 6,
                                                 verbose = 1,
                                                 min_delta = 5*1e-3,
                                                 min_lr = 5*1e-9,)

In [ ]:
# Saving callbacks in a list
callback = [model_save, reduce_lr]

**Model Training**

In [ ]:
# Training the model
history = model.fit(train_datagen,
                    epochs = 15,
                    steps_per_epoch = (len(train_datagen)),
                    validation_data = val_datagen,
                    validation_steps = (len(val_datagen)),
                    shuffle = False,
                    callbacks = callback)

In [ ]:
# Summarize history for loss
tr_losses = history.history['loss']
val_losses = history.history['val_loss']
tr_accs = history.history['accuracy']
val_accs = history.history['val_accuracy']
plt.plot(tr_losses, label = "train_loss")
plt.plot(val_losses, label = "val_loss")
plt.xlabel("Number of epochs")
plt.ylabel("Loss")
plt.grid()
plt.legend()
plt.show()

In [ ]:
# Summarize history for accuracy
plt.plot(tr_accs, label = "acc_train")
plt.plot(val_accs, label = "acc_val")
plt.ylim((0.3, 1.))
plt.xlabel("Number of epochs")
plt.ylabel("Accuracy")
plt.grid()
plt.legend()
plt.show()

**Model Evaluation**

In [ ]:
# Prediction Function
predictions = model.predict(test_datagen, verbose = 1, steps = (len(test_datagen)))

In [ ]:
# Only save class with highest confidence for each image
predictions.squeeze().argmax(axis = 1)

In [ ]:
# Classification Report
print(classification_report(test_datagen.classes, predictions.squeeze().argmax(axis = 1)))

In [ ]:
# Confusion Matrix
print('Confusion Matrix')
cm = confusion_matrix(test_datagen.classes, predictions.squeeze().argmax(axis = 1))

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)

disp = disp.plot(include_values = True,
                 values_format = 'd',
                 cmap = 'viridis',
                 ax = None,
                 xticks_rotation = 'horizontal')

plt.show()

**External Test Images**

In [ ]:
def test(path):

  # Load the image using keras
  img = tf.keras.preprocessing.image.load_img(path,
                                              grayscale = False,
                                              color_mode = 'rgb',
                                              target_size = (224, 224, 3),
                                              interpolation = 'nearest')

  # Display the image
  plt.imshow(img)
  plt.axis('off')

  # Convert image to array for feeding it to the model
  img_array = np.asarray(img)

  # Expand dimension of img array
  img_array = np.expand_dims(img_array, 0)

  # Take prediction
  predictions = model.predict(img_array)

  # Evaluate Score
  score = predictions[0]

  return print('This image is a {} with a {:.2f} % confidence.'.format(classes[np.argmax(score)], 100 * np.max(score)))

In [ ]:
# Download test images
!wget -O test-images.zip "https://www.dropbox.com/sh/2hpc50t687r0fdt/AADokyQsM-0kacFVl5KL-ONTa?dl=1"

# Extract images
!unzip -q -u test-images.zip

In [ ]:
# Testing Image
test('/content/test-image-1.jpg')

In [ ]:
# Testing Image
test('/content/test-image-2.jpg')

In [ ]:
# Testing Image
test('/content/test-image-3.jpg')

In [ ]:
# Testing Image
test('/content/test-image-4.jpg')

In [ ]:
# Testing Image
test('/content/test-image-5.jpg')